# Load the VGG16 Model

In [48]:
from keras.applications.vgg16 import VGG16

model = VGG16(
    weights='imagenet',
    include_top=False
)

# View the names and output shape

In [49]:
for layer in model.layers:
    if 'conv' not in layer.name:
        continue
    print(f'Layer name: {layer.name}, output_shape: {layer.output.shape}')

Layer name: block1_conv1, output_shape: (None, None, None, 64)
Layer name: block1_conv2, output_shape: (None, None, None, 64)
Layer name: block2_conv1, output_shape: (None, None, None, 128)
Layer name: block2_conv2, output_shape: (None, None, None, 128)
Layer name: block3_conv1, output_shape: (None, None, None, 256)
Layer name: block3_conv2, output_shape: (None, None, None, 256)
Layer name: block3_conv3, output_shape: (None, None, None, 256)
Layer name: block4_conv1, output_shape: (None, None, None, 512)
Layer name: block4_conv2, output_shape: (None, None, None, 512)
Layer name: block4_conv3, output_shape: (None, None, None, 512)
Layer name: block5_conv1, output_shape: (None, None, None, 512)
Layer name: block5_conv2, output_shape: (None, None, None, 512)
Layer name: block5_conv3, output_shape: (None, None, None, 512)


# Gradient ascent process:
    - Define a loss function that seeks to maximize the activation of a specific filter (filter _index) in a specific layer (layer_name)

In [50]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np


# 입력 이미지 로드 및 전처리
img_path = 'samoyed.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
# print(img_array.shape)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# 지정된 레이어 및 필터 설정
layer_name = 'block3_conv1'
filter_index = 127

# 그래디언트 테이프 사용
with tf.GradientTape() as tape:
    # 입력 이미지에 대한 텐서 설정
    input_img = tf.convert_to_tensor(img_array)
    tape.watch(input_img) # tape 객체가 input_img 텐서를 추적 # 그래디언트를 계산할 때 필요
    
    # 모델의 출력 얻기
    layer_output = model.get_layer(layer_name).output # KerasTensor
    model_output = tf.keras.Model(inputs=model.inputs, outputs=layer_output)
    layer_output_value = model_output(input_img) # 다양한 레이어 출력에 접근 # EagerTensor
    # print(f'layer_output_value.shape: {layer_output_value.shape}')
    
    # 손실 함수 계산
    loss = tf.reduce_mean(layer_output_value[:, :, :, filter_index])
    # tf.reduce_mean은 주어진 텐서의 평균 값을 계산하는 함수
    # 선택된 필터의 활성화 맵에 대한 평균 값을 계산

# 그래디언트 계산
grads = tape.gradient(
    target=loss,
    sources=input_img
)
# gradient 메서드는 tf.GradientTape 컨텍스트 내에서 기록된 연산을 바탕으로, 주어진 스칼라 텐서(보통 손실 함수)에 대한 하나 이상의 텐서의 그래디언트를 계산

# 그래디언트 정규화
grads /= (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)

# 손실과 그래디언트를 반환하는 함수
iterate = tf.function(lambda x: (loss, grads))
# tf.function: 반복적인 연산을 효율적으로 처리

# 함수 호출
# loss_value, grads_value = iterate(input_img)
# print('Loss:', loss_value.numpy())
# print('Gradients:', grads_value.numpy())

###  tf.GradientTape
    - 텐서의 연산을 기록하고, 이 기록을 바탕으로 그래디언트를 자동으로 계산하는 도구
        - with 블록 내에서 수행된 연산을 모두 기록
    - tf.GradientTape 객체가 텐서로 변환된 img_array를 추적하는 이유는 그래디언트를 계산할 때 이 텐서가 손실 함수에 어떻게 영향을 미치는지 파악하기 위해서
        - 입력 이미지가 특정 레이어와 필터의 활성화를 최대화하기 위해 어떻게 변경되어야 하는지 알 수 있다.

### tape의 주요 기능과 특징
    - 자동 미분: tf.GradientTape는 연산을 기록하고, 이를 바탕으로 미분(그래디언트)을 자동으로 계산
    - 컨텍스트 관리: with 블록 안에서 실행된 모든 연산을 기록
        - with 블록이 끝나면, tape 객체는 기록된 연산을 바탕으로 그래디언트를 계산
    - 다양한 연산 지원: 텐서의 다양한 연산을 지원하며, 특히 딥러닝 모델의 학습 과정에서 손실 함수에 대한 그래디언트를 계산할 때 유용
    - 위의 코드에서는 input_img에 대한 손실 함수 loss의 그래디언트를 계산하기 위해 tape 객체를 사용
        - 이를 통해 입력 이미지가 특정 레이어와 필터의 활성화를 최대화하도록 조정

### 그래디언트 계산의 의미
    - 손실 함수에 대한 입력 이미지의 그래디언트를 계산함으로써 입력 이미지를 수정할 방향을 알 수 있다.

# Do gradient ascent

In [54]:
def gradient_ascent(n_times, input_img, step):
    for _ in range(n_times):
        loss_value, grads_value = iterate([input_img])
        input_img += grads_value * step

# Convert tensor into a valid image

In [57]:
from PIL import Image

def deprocess_image(x):
    # # Real Image
    # x = x.numpy()  # EagerTensor를 NumPy 배열로 변환

    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    
    x += 0.5
    x = np.clip(x, 0, 1)
    
    # Random Data
    x *= 255
    x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8') # Converts to an RGB array
    
    # # Real image
    # x *= 255
    # x = x[0]  # (1, 224, 224, 3)에서 (224, 224, 3)으로 변환
    # x = x.astype('uint8')  # uint8로 변환
    
    return x

# 이미지 크기 조정
def resize_image(img, scale_factor):
    width, height = img.size
    new_size = (int(width * scale_factor), int(height * scale_factor))
    return img.resize(new_size, Image.LANCZOS)

step = 0.1
n_times = 20
# gradient_ascent(n_times,input_img,step)
# 
# img = deprocess_image(input_img)
# img = Image.fromarray(img)
# img = resize_image(img, 2)  # 두 배 크기로 조정
# img.save('%s_filter_%d_large.png' % (layer_name, filter_index))

## Random Data

In [66]:
import imageio

img_width = 224
img_height = 224

input_img_data = np.random.random((1, img_width, img_height, 3)) * 20 + 128
gradient_ascent(n_times, input_img_data, step)
random_img = input_img_data[0]
random_img = deprocess_image(random_img)
imageio.imwrite('%s_filter_%d.png' % (layer_name, filter_index), random_img)

(150528, 1)


ValueError: axes don't match array